In [29]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [34]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])

shadow_cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
shadow_robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), shadow_cfg)
vm_cfg = URDFParserConfig(;suppress_warnings=true) 
# For the moment the urdfs are the same but we might want to change the properties of the virtual robot
vm_robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), vm_cfg) 

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [35]:
add_gravity_compensation!(vm_robot, VMRobotControl.DEFAULT_GRAVITY)

joint_limits = vm_cfg.joint_limits

for joint_id in keys(joints(vm_robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(vm_robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(vm_robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(vm_robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end


### Addition of the multiple springs/dampers

In [36]:
vms = VirtualMechanismSystem("myShadowVMS", shadow_robot, vm_robot)

# MOTION MECHANISMS
add_coordinate!(vms, FrameOrigin(".virtual_mechanism.rh_fftip"); id="fftip position")
add_coordinate!(vms, FrameOrigin(".virtual_mechanism.rh_palm"); id="palm position")
add_coordinate!(vms, CoordDifference("fftip position", "palm position"); id="fftip position error")

add_component!(vms, LinearSpring(50.0, "fftip position error"); id="fftip position spring")
add_component!(vms, LinearDamper(5.0, "fftip position error"); id="fftip position damper")

joint_limits = shadow_cfg.joint_limits

for joint_id in keys(joints(shadow_robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(shadow_robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    add_coordinate!(vms, CoordDifference(".robot.$(joint_id)_coord", ".virtual_mechanism.$(joint_id)_coord");id="$(joint_id) coord diff")
    add_component!(vms, LinearSpring(0.1, "$(joint_id) coord diff"); id = "$(joint_id) coord spring")
    add_component!(vms, LinearDamper(0.01, "$(joint_id) coord diff"); id = "$(joint_id) coord damper")
end


In [37]:
cvms = compile(vms)

CompiledVirtualMechanismSystem{Float64, ...}(myShadowVMS...)

## Simulating the Robot

### Setting Up the Simulation

In [6]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

"""
Update the vms based on the "coord", which is a coordinate varying between 
0 and 1, 0 being the starting position of the lateral pinch motion while 1 being 
the end
"""
function update_power_sphere_coord(args, cache, coord)
    nothing
end

function f_setup(cache) 
    nothing
    # return  (get_compiled_coordID(cache, "angular spring length"), get_compiled_coordID(cache, "th spring length"), get_compiled_coordID(cache, "J3 target angle"),
    #         get_compiled_coordID(cache, "J2 target angle"), get_compiled_coordID(cache, "J1 target angle"), get_compiled_coordID(cache, "th J5 target angle"),
    #         get_compiled_coordID(cache, "th J2 target angle"), get_compiled_coordID(cache, "th J1 target angle"))
end

function f_control(cache, t, args, extra)

    # t_start = 0.
    # t_end = 4. #we want to reach the end position at t = t_end 
    
    # if t >= t_start && t <= t_end 
    #     coord_value = (t - t_start) /(t_end -t_start)
    #     update_power_sphere_coord(args, cache, coord_value)
    # end
    nothing
end

tspan = (0., 5.)
vms_compiled = compile(vms)

q = (zero_q(vms_compiled.robot), zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot to make some tests"

sol = solve(prob, Rosenbrock23(autodiff=false), progress=true; maxiters=1e6, abstol=1e-4, reltol=1e-4);

[ Info: Simulating shadow robot to make some tests
ODE   0%|█                                              |  ETA: N/A
ODE 100%|███████████████████████████████████████████████| Time: 0:00:02


### Visualizing the Results

In [29]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_tests.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);

In [ ]:
joint_names = ["rh_WRJ1", "rh_WRJ2", "rh_FFJ1", "rh_FFJ2", "rh_FFJ3", "rh_FFJ4", "rh_MFJ1",
                "rh_MFJ2", "rh_MFJ3", "rh_MFJ4", "rh_RFJ1", "rh_RFJ2", "rh_RFJ3", "rh_RFJ4", 
                "rh_LFJ1", "rh_LFJ2", "rh_LFJ3", "rh_LFJ4", "rh_LFJ5", "rh_THJ1", "rh_THJ2", 
                "rh_THJ3", "rh_THJ4", "rh_THJ5"]

function generate_virtual_robot_idxs(vms_compiled, ordered_joint_names)
    robot_idxs = []

    for joint_name in ordered_joint_names
        joint_id = get_compiled_jointID(vms_compiled.virtual_mechanism, joint_name)
        joint = vms_compiled.virtual_mechanism[joint_id]
        joint_idx = q_idxs(joint)
        push!(robot_idxs, joint_idx[1])
    end

    return robot_idxs
end

conversion_list = generate_virtual_robot_idxs(vms_compiled, joint_names)

qr, qv = get_q(dcache)

function get_virtual_hand_state(q_vm, idxs)
    hand_state = []
    for idx in idxs
        push!(hand_state, q_vm[idx])
    end 
    return hand_state
end

hand_state = get_virtual_hand_state(qv, conversion_list)
display(hand_state)

24-element Vector{Any}:
 -0.00020660406593791774
 -5.5005491840259505e-5
  0.16705569376630222
  0.2513085783178839
  0.002077724918037598
  1.0715007862990576e-5
  0.16700021902800208
  0.2512055862214286
  0.0022134859100442134
  1.1136773430899554e-5
  0.16705249718606965
  0.2513020148594966
  0.002092635228933274
 -1.0990282475013898e-5
  0.2167196631625505
  0.10056490893996937
 -0.05776378894500269
  0.5242856928186829
 -0.002281632736553128
 -0.008936111440931396
  0.10943951038094998
  0.5963359179926188
  0.005077856175425158
  1.0000000133567926